In [130]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import sys
sys.path.append("..")
from src import sp_analisis as sa
pd.set_option('display.max_columns', None)

# Ejercicio 4. Querys avanzadas 
#### Nuestro jefe acaba de venir y nos ha hecho una serie de peticiones sobre la base de datos que tenemos que poder contestar.
- #### Quiere saber cuándo fue la última vez que se pidió un producto de cada categoría.

In [2]:
df_details_prod_supl = pd.read_csv('../data/details_prod_supl.csv')
query_categories =''' SELECT "category_id",
                              "category_name",
                              "description"
                          FROM "categories";
'''
df_categories = sa.pasos_querys(query_categories)
df_prod_cat = pd.merge( df_details_prod_supl,
                       df_categories,
                       how='inner',
                       on= "category_id"
)
df_prod_cat.sample()

,product_id,supplier_id,category_id,product_name,unit_price_product,units_in_stock,units_on_order,discontinued,order_id,quantity,unit_price_order,company_name,city,country,category_name,description
92,3,1,2,Aniseed Syrup,10.0,13,70,0,11017,25,10.0,Exotic Liquids,London,UK,Condiments,"Sweet and savory sauces, relishes, spreads, an..."


In [8]:
df_orders = pd.read_csv('../data/orders.csv')
df_total = pd.merge( df_orders,
                     df_prod_cat,
                     how='inner',
                     on= "order_id")
df_total.columns

Index(['order_id', 'customer_id', 'ship_via', 'order_date', 'required_date',
       'shipped_date', 'product_id', 'supplier_id', 'category_id',
       'product_name', 'unit_price_product', 'units_in_stock',
       'units_on_order', 'discontinued', 'quantity', 'unit_price_order',
       'company_name', 'city', 'country', 'category_name', 'description'],
      dtype='object')

In [16]:
df_ult_fecha_cat = df_total.groupby(['category_id', 'category_name'])['order_date'].max().reset_index()
df_ult_fecha_cat

,category_id,category_name,order_date
0,1,Beverages,1998-05-06
1,2,Condiments,1998-05-06
2,3,Confections,1998-05-06
3,4,Dairy Products,1998-05-06
4,5,Grains/Cereals,1998-05-06
5,6,Meat/Poultry,1998-05-06
6,7,Produce,1998-05-06
7,8,Seafood,1998-05-06


- #### Necesita saber si existe algún producto que nunca se haya vendido por su precio original.

In [38]:
df_dif_price= df_prod_cat[df_prod_cat['unit_price_product'] != df_prod_cat['unit_price_order']]
print(f'Hay {df_dif_price.shape[0]} productos que no se han vendido por su precio original')


Hay 662 productos que no se han vendido por su precio original


- #### Quiere tener toda la información necesaria para poder identificar un tipo de producto. En concreto, tienen especial interés por los productos con categoría "Confections". Devuelve el ID del producto, el nombre del producto y su ID de categoría.

In [59]:
df_confections = df_prod_cat[df_prod_cat['category_name'].str.contains("Confections", na=False)] [['product_id', 'product_name', 'category_id', 'category_name']].drop_duplicates()
df_confections

,product_id,product_name,category_id,category_name
336,16,Pavlova,3,Confections
443,19,Teatime Chocolate Biscuits,3,Confections
480,20,Sir Rodney's Marmalade,3,Confections
496,21,Sir Rodney's Scones,3,Confections
620,25,NuNuCa Nuß-Nougat-Creme,3,Confections
638,26,Gumbär Gummibärchen,3,Confections
670,27,Schoggi Schokolade,3,Confections
1231,47,Zaanse koeken,3,Confections
1252,48,Chocolade,3,Confections
1258,49,Maxilaku,3,Confections


- #### Quiere saber si existe algún proveedor del que pueda prescindir ya que todos los productos que tiene se encuentran descontinuados.

      Como podemos ver, hay varios productos descontinuados, y la compañia que tiene más productos descontinuados es la 12 (Plutzer Lebensmittelgroßmärkte AG) pero son solo 2 productos descontinuados y provee otros 3 productos que no están descontinuados*

In [69]:
df_disc_prod= df_details_prod_supl.loc[df_details_prod_supl['discontinued'] == 1, ['supplier_id','company_name','product_id','product_name' ]].drop_duplicates()
df_disc_prod

,supplier_id,company_name,product_id,product_name
0,8,"Specialty Biscuits, Ltd.",1,Chai
38,1,Exotic Liquids,2,Chang
114,2,New Orleans Cajun Delights,5,Chef Anton's Gumbo Mix
178,4,Tokyo Traders,9,Mishi Kobe Niku
379,7,"Pavlova, Ltd.",17,Alice Mutton
569,10,Refrescos Americanas LTDA,24,Guaraná Fantástica
679,12,Plutzer Lebensmittelgroßmärkte AG,28,Rössle Sauerkraut
712,12,Plutzer Lebensmittelgroßmärkte AG,29,Thüringer Rostbratwurst
1108,20,Leka Trading,42,Singaporean Hokkien Fried Mee
1357,24,"G'day, Mate",53,Perth Pasties


In [75]:
df_details_prod_supl.loc[df_details_prod_supl['supplier_id'] == 12,['product_name','product_id']].drop_duplicates()

,product_name,product_id
679,Rössle Sauerkraut,28
712,Thüringer Rostbratwurst,29
1741,Wimmers gute Semmelknödel,64
2032,Rhönbräu Klosterbier,75
2117,Original Frankfurter grüne Soße,77


- #### Extraer los clientes que compraron mas de 30 articulos "Chai" en un único pedido

In [82]:
df_products= pd.read_csv('../data/products.csv')
df_products[df_products['product_name'].str.contains(pat = "Chai")]

,product_id,supplier_id,category_id,product_name,unit_price,units_in_stock,units_on_order,discontinued
0,1,8,1,Chai,18.0,39,0,1


In [121]:
df_cust_orders = pd.read_csv('../data/customers_orders.csv')
query_details = ''' SELECT "order_id",
                            "product_id",
                            "unit_price",
                            "quantity"
                     FROM "order_details";
'''
df_details = sa.pasos_querys(query_details)
df_details_cust_ord = pd.merge( df_cust_orders,
                       df_details,
                       how='inner',
                       on= "order_id"
)
df_details_cust_ord.sample()

,customer_id,company_name,city,country,order_id,ship_via,order_date,required_date,shipped_date,continent,days_to_ship,delays,product_id,unit_price,quantity
198,BONAP,Bon app',Marseille,France,10871.0,2.0,1998-02-05,1998-03-05,1998-02-10,Europe,5.0,False,16,17.45,12


In [87]:
df_chai= df_details_cust_ord[(df_details_cust_ord['product_id'] == 1) &( df_details_cust_ord['quantity'] > 30)]
df_chai

,customer_id,company_name,city,country,order_id,ship_via,order_date,required_date,shipped_date,continent,days_to_ship,delays,product_id,unit_price,quantity
100,BERGS,Berglunds snabbköp,Luleå,Sweden,10689.0,2.0,1997-10-01,1997-10-29,1997-10-07,Europe,6.0,False,1,18.0,35
228,BOTTM,Bottom-Dollar Markets,Tsawassen,Canada,10918.0,3.0,1998-03-02,1998-03-30,1998-03-11,America,9.0,False,1,18.0,60
971,LEHMS,Lehmanns Marktstand,Frankfurt a.M.,Germany,10522.0,1.0,1997-04-30,1997-05-28,1997-05-06,Europe,6.0,False,1,18.0,40
995,LEHMS,Lehmanns Marktstand,Frankfurt a.M.,Germany,11070.0,1.0,1998-05-05,1998-06-02,NaN,Europe,NaN,True,1,18.0,40
1055,LINOD,LINO-Delicateses,I. de Margarita,Venezuela,10729.0,3.0,1997-11-04,1997-12-16,1997-11-14,America,10.0,False,1,18.0,50
1359,QUICK,QUICK-Stop,Cunewalde,Germany,10285.0,2.0,1996-08-20,1996-09-17,1996-08-26,Europe,6.0,False,1,14.4,45
1709,SAVEA,Save-a-lot Markets,Boise,USA,10847.0,3.0,1998-01-22,1998-02-05,1998-02-10,America,19.0,True,1,18.0,80
1738,SAVEA,Save-a-lot Markets,Boise,USA,11031.0,2.0,1998-04-17,1998-05-15,1998-04-24,America,7.0,False,1,18.0,45
1770,SEVES,Seven Seas Imports,London,UK,10869.0,1.0,1998-02-04,1998-03-04,1998-02-09,Europe,5.0,False,1,18.0,40


- #### Indica los clientes cuya suma total de carga en los pedidos sea mayor de 1000

In [129]:
df_details_cust_ord['total_order'] = df_details_cust_ord['unit_price'] * df_details_cust_ord['quantity']
df_plus= df_details_cust_ord[df_details_cust_ord['total_order'] > 1000].reset_index()
df_plus

,index,customer_id,company_name,city,country,order_id,ship_via,order_date,required_date,shipped_date,continent,days_to_ship,delays,product_id,unit_price,quantity,total_order
0,25,ANTON,Antonio Moreno Taquería,México D.F.,Mexico,10535.0,1.0,1997-05-13,1997-06-10,1997-05-21,America,8.0,False,11,21.0,50,1050.0
1,47,AROUT,Around the Horn,London,UK,10558.0,2.0,1997-06-04,1997-07-02,1997-06-10,Europe,6.0,False,51,53.0,20,1060.0
2,65,AROUT,Around the Horn,London,UK,10953.0,2.0,1998-03-16,1998-03-30,1998-03-25,Europe,9.0,False,20,81.0,50,4050.0
3,76,BERGS,Berglunds snabbköp,Luleå,Sweden,10384.0,3.0,1996-12-16,1997-01-13,1996-12-20,Europe,4.0,False,20,64.8,28,1814.4
4,86,BERGS,Berglunds snabbköp,Luleå,Sweden,10524.0,2.0,1997-05-01,1997-05-29,1997-05-07,Europe,6.0,False,43,46.0,60,2760.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2109,WHITC,White Clover Markets,Seattle,USA,10861.0,2.0,1998-01-30,1998-02-27,1998-02-17,America,18.0,False,17,39.0,42,1638.0
346,2110,WHITC,White Clover Markets,Seattle,USA,10861.0,2.0,1998-01-30,1998-02-27,1998-02-17,America,18.0,False,18,62.5,20,1250.0
347,2115,WHITC,White Clover Markets,Seattle,USA,10904.0,3.0,1998-02-24,1998-03-24,1998-02-27,America,3.0,False,62,49.3,35,1725.5
348,2117,WHITC,White Clover Markets,Seattle,USA,11032.0,3.0,1998-04-17,1998-05-15,1998-04-23,America,6.0,False,38,263.5,25,6587.5


- #### Desde recursos humanos nos piden seleccionar los nombres de las ciudades con 5 o más empleadas de cara a estudiar la apertura de nuevas oficinas

      Las tiendas tienen un empleado y hay algunas tiendas que no tienen un empleado asociado. Otra cosa seria si tuvieramos que decir cuantas tiendas tienen cada empleado a su cargo, como se puede ver en el ultimo dataframe.

In [120]:
query_territories = ''' SELECT "territory_id","territory_description"
                        FROM "territories"
                    '''
df_territories = sa.pasos_querys(query_territories)
query_empl_terr = ''' SELECT "employee_id", "territory_id" 
                     FROM "employee_territories"'''
df_empl_terr = sa.pasos_querys(query_empl_terr)
df_cities = pd.merge(df_territories, 
                     df_empl_terr,
                     how='left', 
                     on= "territory_id")

In [118]:
df_store= df_cities.groupby('territory_id')['employee_id'].count().reset_index().sort_values('employee_id',ascending=False)
df_store

,territory_id,employee_id
0,01581,1
40,80202,1
27,44122,1
28,45839,1
29,48075,1
30,48084,1
31,48304,1
32,53404,1
33,55113,1
34,55439,1


In [119]:
df_cities.groupby('employee_id')['territory_description'].count().reset_index()

,employee_id,territory_description
0,1.0,2
1,2.0,7
2,3.0,4
3,4.0,3
4,5.0,7
5,6.0,5
6,7.0,10
7,8.0,4
8,9.0,7
